# Day 2: Password Philosophy
## Part 1:
Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan. The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen. To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:
* 1-3 a: abcde
* 1-3 b: cdefg
* 2-9 c: ccccccccc

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

How many passwords are valid according to their policies?

In [1]:
import pandas as pd

In [22]:
# read in the input file
df = pd.read_table('data/day2.txt', sep='\s', names=('range', 'char', 'password'))
df.head()

/Users/amandapogue/anaconda3/envs/interviews/lib/python3.7/site-packages/pandas/io/parsers.py:755: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


,range,char,password
0,1-4,n:,nnnnn
1,5-7,z:,qhcgzzz
2,7-11,m:,mmmmmmsmmmmm
3,5-8,d:,ldddtdgnzddddwl
4,16-18,q:,qsqqqqqqqqqpqqqlqhq


In [28]:
df[['lower','upper']] = df.range.str.split('-', expand=True)
df['char'] = df.char.str.strip(':')

In [145]:
def password_check(char, lower, upper, password):
    '''
    takes the repeated password character, the lower and upper bounds, and the password 
    itself and returns a 1 if the password is valid
    '''
    count = 0
    for i in password:
        if i == char:
            count += 1        
    return count
        

In [146]:
# check the password
df['count'] = df.apply(lambda row: password_check(row['char'], row['lower'], row['upper'], row['password']), axis=1)

In [153]:
df['check'] = df.apply(lambda row: 1 if (row['count'] >= int(row['lower'])) & (row['count'] <= int(row['upper'])) else 0, axis=1)

In [154]:
print(f"There are {df.check.value_counts()[1]} valid passwords")

There are 456 valid passwords


## Part Two
While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting. The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

* 1-3 a: abcde is valid: position 1 contains a and position 3 does not.
* 1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
* 2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.
How many passwords are valid according to the new interpretation of the policies?

In [156]:
def new_policy(char, lower, upper, password):
    pos_1 = int(lower)-1
    pos_2 = int(upper)-1
    if bool(password[pos_1]==char) != bool(password[pos_2]==char):
        return "valid"

In [157]:
df['part_2'] = df.apply(lambda row: new_policy(row['char'], row['lower'], row['upper'], row['password']), axis=1)

In [158]:
df.part_2.value_counts()

valid    308
Name: part_2, dtype: int64

In [159]:
df.head()

,range,char,password,lower,upper,check,count,part_2
0,1-4,n,nnnnn,1,4,0,5,None
1,5-7,z,qhcgzzz,5,7,0,3,None
2,7-11,m,mmmmmmsmmmmm,7,11,1,11,valid
3,5-8,d,ldddtdgnzddddwl,5,8,1,8,None
4,16-18,q,qsqqqqqqqqqpqqqlqhq,16,18,0,15,None
